In [2]:
from transformers import GPT2LMHeadModel

/Users/mrigank/miniconda3/envs/torch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = GPT2LMHeadModel.from_pretrained('gpt2') # gpt2-medium, gpt2-large, gpt2-xl

In [5]:
sd_hf = model.state_dict()
for k, v in sd_hf.items():
    print(f'{k} --> {v.shape}')

transformer.wte.weight --> torch.Size([50257, 768])
transformer.wpe.weight --> torch.Size([1024, 768])
transformer.h.0.ln_1.weight --> torch.Size([768])
transformer.h.0.ln_1.bias --> torch.Size([768])
transformer.h.0.attn.c_attn.weight --> torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias --> torch.Size([2304])
transformer.h.0.attn.c_proj.weight --> torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias --> torch.Size([768])
transformer.h.0.ln_2.weight --> torch.Size([768])
transformer.h.0.ln_2.bias --> torch.Size([768])
transformer.h.0.mlp.c_fc.weight --> torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias --> torch.Size([3072])
transformer.h.0.mlp.c_proj.weight --> torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias --> torch.Size([768])
transformer.h.1.ln_1.weight --> torch.Size([768])
transformer.h.1.ln_1.bias --> torch.Size([768])
transformer.h.1.attn.c_attn.weight --> torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias --> torch.Size([2304])
transformer.h.1.att

In [ ]:
# import tiktoken
from transformers import GPT2Tokenizer
# enc = tiktoken.get_encoding('gpt2')


In [ ]:
prompt = "Hi, I'm a language model,"
# tokens = enc.encode(prompt)
# tokens

In [ ]:
prompt = "Hi, I'm a language model,"
tok = GPT2Tokenizer.from_pretrained('gpt2')
input_ids = tok.encode(prompt, return_tensors='pt')
input_ids

In [ ]:
output_ids = model.generate(input_ids, max_new_tokens=30)
output_ids

In [ ]:
tok.decode(output_ids[0])

## causal self attention elaborate affinity scores

In [3]:
import torch

In [92]:
b,t,c = 2,3,4
nh,hs = 2,2
qkv = torch.randn((b,t,3*c))
q,k,v = qkv.split(c, dim=-1)

In [93]:
torch.allclose(qkv[:,:,:c], q)

True

In [94]:
q.shape, q.view(b,t,nh,hs).shape, q.view(b,t,nh,hs).transpose(1,2).shape

(torch.Size([2, 3, 4]), torch.Size([2, 3, 2, 2]), torch.Size([2, 2, 3, 2]))

In [95]:
q.view(b,nh,t,hs)

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [96]:
q = q.view(b,t,nh,hs).transpose(1,2)
k = k.view(b,t,nh,hs).transpose(1,2)
v = v.view(b,t,nh,hs).transpose(1,2)

affinity_scores = (q @ k.transpose(-2,-1)) # (b,nh,t,t)
affinity_scores.shape

torch.Size([2, 2, 3, 3])

In [24]:
tril = torch.tril(torch.ones(t,t, dtype=torch.long))
tril # (t,t)

tensor([[1, 0, 0],
        [1, 1, 0],
        [1, 1, 1]])

In [105]:
affinity_scores = (q @ k.transpose(-2,-1)) # (b,nh,t,t)
tril = torch.tril(torch.ones(t,t, dtype=torch.long))

affinity_scores = affinity_scores.masked_fill(tril==0, float('-inf'))
affinity_scores = torch.nn.functional.softmax(affinity_scores, dim=-1)
affinity_scores # (b,nh,t,t)

tensor([[[[1.0000, 0.0000, 0.0000],
          [0.9927, 0.0073, 0.0000],
          [0.7182, 0.0590, 0.2228]],

         [[1.0000, 0.0000, 0.0000],
          [0.5729, 0.4271, 0.0000],
          [0.3929, 0.3388, 0.2683]]],


        [[[1.0000, 0.0000, 0.0000],
          [0.9259, 0.0741, 0.0000],
          [0.9809, 0.0021, 0.0169]],

         [[1.0000, 0.0000, 0.0000],
          [0.5949, 0.4051, 0.0000],
          [0.1191, 0.4222, 0.4587]]]])

In [106]:
y = affinity_scores @ v # (b,nh,t,t) @ (b,nh,t,hs) -> (b,nh,t,hs)
y.shape

torch.Size([2, 2, 3, 2])

In [107]:
y

tensor([[[[ 0.1593,  0.3919],
          [ 0.1635,  0.3851],
          [-0.0017,  0.3043]],

         [[-0.8028,  0.3271],
          [-0.3842, -0.0507],
          [-0.1121,  0.0312]]],


        [[[-0.0119,  0.1332],
          [ 0.0475,  0.1994],
          [-0.0344,  0.1170]],

         [[-1.2762,  1.7055],
          [-1.0988,  1.5696],
          [-0.3748,  0.2845]]]])

In [108]:
y.view(b,t,c)

tensor([[[ 0.1593,  0.3919,  0.1635,  0.3851],
         [-0.0017,  0.3043, -0.8028,  0.3271],
         [-0.3842, -0.0507, -0.1121,  0.0312]],

        [[-0.0119,  0.1332,  0.0475,  0.1994],
         [-0.0344,  0.1170, -1.2762,  1.7055],
         [-1.0988,  1.5696, -0.3748,  0.2845]]])

In [109]:
y = y.transpose(1,2).contiguous().view(b,t,c)
y

tensor([[[ 0.1593,  0.3919, -0.8028,  0.3271],
         [ 0.1635,  0.3851, -0.3842, -0.0507],
         [-0.0017,  0.3043, -0.1121,  0.0312]],

        [[-0.0119,  0.1332, -1.2762,  1.7055],
         [ 0.0475,  0.1994, -1.0988,  1.5696],
         [-0.0344,  0.1170, -0.3748,  0.2845]]])

In [103]:
y

tensor([[[ 0.1593,  0.3919,  0.1635,  0.3851],
         [-0.0017,  0.3043, -0.8028,  0.3271],
         [-0.3842, -0.0507, -0.1121,  0.0312]],

        [[-0.0119,  0.1332,  0.0475,  0.1994],
         [-0.0344,  0.1170, -1.2762,  1.7055],
         [-1.0988,  1.5696, -0.3748,  0.2845]]])